# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy.stats import percentileofscore as score

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

symbol = 'AAPL'
base_url = 'https://sandbox.iexapis.com/stable'
api_url = f"{base_url}/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2505794871536, 'week52high': 157.14, 'week52low': 104.72, 'week52highSplitAdjustOnly': 156.4, 'week52lowSplitAdjustOnly': 105.9, 'week52change': 0.2084980026251539, 'sharesOutstanding': 17304726240, 'float': 0, 'avg10Volume': 84272954, 'avg30Volume': 72624512, 'day200MovingAvg': 133.49, 'day50MovingAvg': 147.71, 'employees': 147891, 'ttmEPS': 5.1, 'ttmDividendRate': 0.8881700875854064, 'dividendYield': 0.005971154686864102, 'nextDividendDate': '', 'exDividendDate': '2021-07-30', 'nextEarningsDate': '2021-10-14', 'peRatio': 28.980195998765115, 'beta': 1.53510380725446, 'maxChangePercent': 58.47450683570732, 'year5ChangePercent': 4.795304364500271, 'year2ChangePercent': 1.9154616787085228, 'year1ChangePercent': 0.2690329241842109, 'ytdChangePercent': 0.1281634009235089, 'month6ChangePercent': 0.15014244467407484, 'month3ChangePercent': 0.16778066236134676, 'month1ChangePercent': 0.01585179899926691, 'day30ChangePercent': 0.020716950177260197, 'da

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.2690329241842109

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

# for symbol_string in symbol_strings:
#     print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
def momentum_data():
    dataframe = pd.DataFrame(columns = my_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        data[symbol]['stats']['year1ChangePercent'],
                        buy_count
                    ],
                    index = my_columns
                ),
                ignore_index = True
            )

    return dataframe

final_dataframe = momentum_data()
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,170.83,0.746864,N/A
1,AAL,18.90,0.49,N/A
2,AAP,218.25,0.356944,N/A
3,AAPL,151.06,0.262722,N/A
4,ABBV,119.09,0.313726,N/A
...,...,...,...,...
500,YUM,134.55,0.452623,N/A
501,ZBH,149.50,0.070751,N/A
502,ZBRA,573.67,1.084923,N/A
503,ZION,55.64,0.727933,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50] # Dump all but the first 50
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,80.23,2.348364,N/A
1,272,KSS,58.96,2.112968,N/A
2,441,TPR,42.28,1.90247,N/A
3,344,NUE,118.54,1.696312,N/A
4,106,COF,175.70,1.644972,N/A
5,129,DFS,128.18,1.629948,N/A
6,148,DVN,26.30,1.527138,N/A
7,23,ALB,229.50,1.504504,N/A
8,253,IVZ,24.79,1.502316,N/A
9,453,UAA,24.58,1.444369,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size

    portfolio_size = 1000000 # input('Enter the value of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print('Portfolio size should be an integer.')
        portfolio_size = input('Enter the value of your portfolio:')
        float(portfolio_size)

portfolio_input()
portfolio_size

1000000

In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    buy_count =  math.floor(position_size / final_dataframe.loc[i, 'Price'])
    final_dataframe.loc[i, 'Number of Shares to Buy'] = buy_count

final_dataframe

/opt/homebrew/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,275,LB,80.23,2.348364,249
1,272,KSS,58.96,2.112968,339
2,441,TPR,42.28,1.90247,473
3,344,NUE,118.54,1.696312,168
4,106,COF,175.70,1.644972,113
5,129,DFS,128.18,1.629948,156
6,148,DVN,26.30,1.527138,760
7,23,ALB,229.50,1.504504,87
8,253,IVZ,24.79,1.502316,806
9,453,UAA,24.58,1.444369,813


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

def hq_momentum_data():
    dataframe = pd.DataFrame(columns = hqm_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f"{base_url}/stock/market/batch?symbols={symbols}&types=stats,price&token={IEX_CLOUD_API_TOKEN}"
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['price']
            # year1ChangePercent = data[symbol]['stats']['year1ChangePercent'],
            buy_count = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        buy_count,
                        data[symbol]['stats']['year1ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month6ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month3ChangePercent'], 
                        np.nan,
                        data[symbol]['stats']['month1ChangePercent'], 
                        np.nan,
                        np.nan,
                    ],
                    index = hqm_columns,
                    
                ),
                ignore_index = True
            )

    # https://stackoverflow.com/questions/65174575/typeerror-not-supported-between-instances-of-nonetype-and-float
    return dataframe.fillna(0)

hqm_dataframe = hq_momentum_data()
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,169.86,0.0,0.761516,0.0,0.333329,0.0,0.276127,0.0,0.131055,0.0,0.0
1,AAL,19.20,0.0,0.490000,0.0,-0.010074,0.0,-0.189892,0.0,-0.104215,0.0,0.0
2,AAP,214.32,0.0,0.351219,0.0,0.274672,0.0,0.072815,0.0,-0.002715,0.0,0.0
3,AAPL,154.49,0.0,0.263169,0.0,0.146158,0.0,0.166587,0.0,0.015993,0.0,0.0
4,ABBV,122.93,0.0,0.314649,0.0,0.159172,0.0,0.027085,0.0,0.024906,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.95,0.0,0.444970,0.0,0.297730,0.0,0.130023,0.0,0.163122,0.0,0.0
501,ZBH,148.07,0.0,0.070271,0.0,-0.068970,0.0,-0.128972,0.0,-0.066906,0.0,0.0
502,ZBRA,597.04,0.0,1.108532,0.0,0.169179,0.0,0.158451,0.0,0.102271,0.0,0.0
503,ZION,55.62,0.0,0.735163,0.0,0.057150,0.0,-0.049404,0.0,0.082557,0.0,0.0


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        column_return = f'{time_period} Price Return'
        column_percentile = f'{time_period} Return Percentile'

        # print(hqm_dataframe.convert_dtypes().dtypes)
        # print(hqm_dataframe[column_return])
        price_return_all = pd.to_numeric(hqm_dataframe[column_return].astype(float))
        price_return_single = hqm_dataframe.loc[row, column_return]

        percentile = score(price_return_all, price_return_single) / 100
        hqm_dataframe.loc[row, column_percentile] = percentile

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,169.86,0.0,0.761516,85.940594,0.333329,88.514851,0.276127,95.445545,0.131055,93.861386,0.0
1,AAL,19.20,0.0,0.490000,64.356436,-0.010074,17.821782,-0.189892,4.356436,-0.104215,3.564356,0.0
2,AAP,214.32,0.0,0.351219,45.544554,0.274672,79.405941,0.072815,66.336634,-0.002715,30.891089,0.0
3,AAPL,154.49,0.0,0.263169,33.861386,0.146158,53.267327,0.166587,85.940594,0.015993,42.772277,0.0
4,ABBV,122.93,0.0,0.314649,41.188119,0.159172,57.029703,0.027085,51.881188,0.024906,47.326733,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.95,0.0,0.444970,59.405941,0.297730,82.376238,0.130023,78.811881,0.163122,96.633663,0.0
501,ZBH,148.07,0.0,0.070271,13.069307,-0.068970,10.099010,-0.128972,10.495050,-0.066906,11.287129,0.0
502,ZBRA,597.04,0.0,1.108532,95.841584,0.169179,60.000000,0.158451,84.356436,0.102271,87.920792,0.0
503,ZION,55.62,0.0,0.735163,83.960396,0.057150,33.267327,-0.049404,29.306931,0.082557,81.980198,0.0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
        hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,169.86,0.0,0.761516,85.940594,0.333329,88.514851,0.276127,95.445545,0.131055,93.861386,90.940594
1,AAL,19.20,0.0,0.490000,64.356436,-0.010074,17.821782,-0.189892,4.356436,-0.104215,3.564356,22.524752
2,AAP,214.32,0.0,0.351219,45.544554,0.274672,79.405941,0.072815,66.336634,-0.002715,30.891089,55.544554
3,AAPL,154.49,0.0,0.263169,33.861386,0.146158,53.267327,0.166587,85.940594,0.015993,42.772277,53.960396
4,ABBV,122.93,0.0,0.314649,41.188119,0.159172,57.029703,0.027085,51.881188,0.024906,47.326733,49.356436
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.95,0.0,0.444970,59.405941,0.297730,82.376238,0.130023,78.811881,0.163122,96.633663,79.306931
501,ZBH,148.07,0.0,0.070271,13.069307,-0.068970,10.099010,-0.128972,10.495050,-0.066906,11.287129,11.237624
502,ZBRA,597.04,0.0,1.108532,95.841584,0.169179,60.000000,0.158451,84.356436,0.102271,87.920792,82.029703
503,ZION,55.62,0.0,0.735163,83.960396,0.057150,33.267327,-0.049404,29.306931,0.082557,81.980198,57.128713


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,IT,316.00,0.0,1.387792,98.019802,0.669909,99.405941,0.313598,97.425743,0.183186,98.217822,98.267327
1,ALB,223.25,0.0,1.455055,98.217822,0.417486,96.039604,0.408411,98.811881,0.179069,98.019802,97.772277
2,FTNT,301.24,0.0,1.267926,97.029703,0.761687,99.603960,0.409535,99.009901,0.132151,94.257426,97.475248
3,NUE,118.94,0.0,1.653172,99.207921,1.064863,100.000000,0.157967,84.158416,0.284053,100.000000,95.841584
4,CARR,56.98,0.0,0.963206,93.069307,0.489745,98.811881,0.277136,95.643564,0.110204,90.099010,94.405941
5,WST,468.82,0.0,0.641025,78.613861,0.611755,99.207921,0.359907,98.019802,0.208253,99.207921,93.762376
6,LLY,283.63,0.0,0.833741,88.712871,0.365179,91.089109,0.362392,98.217822,0.163996,96.831683,93.712871
7,LB,80.74,0.0,2.336642,100.000000,0.832176,99.801980,0.219641,91.881188,0.082858,82.574257,93.564356
8,NVDA,210.15,0.0,0.730728,83.762376,0.410424,95.049505,0.440321,99.603960,0.120412,91.683168,92.524752
9,IPG,37.92,0.0,1.156691,96.633663,0.471917,98.217822,0.113886,76.435644,0.166239,97.227723,92.128713


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()
portfolio_size

1000000

In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for row in hqm_dataframe.index:
    price = hqm_dataframe.loc[row, 'Price']
    buy_count = math.floor(position_size / price)
    hqm_dataframe.loc[row, 'Number of Shares to Buy'] = 0

hqm_dataframe

/opt/homebrew/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,IT,316.00,0.0,1.387792,98.019802,0.669909,99.405941,0.313598,97.425743,0.183186,98.217822,98.267327
1,ALB,223.25,0.0,1.455055,98.217822,0.417486,96.039604,0.408411,98.811881,0.179069,98.019802,97.772277
2,FTNT,301.24,0.0,1.267926,97.029703,0.761687,99.603960,0.409535,99.009901,0.132151,94.257426,97.475248
3,NUE,118.94,0.0,1.653172,99.207921,1.064863,100.000000,0.157967,84.158416,0.284053,100.000000,95.841584
4,CARR,56.98,0.0,0.963206,93.069307,0.489745,98.811881,0.277136,95.643564,0.110204,90.099010,94.405941
5,WST,468.82,0.0,0.641025,78.613861,0.611755,99.207921,0.359907,98.019802,0.208253,99.207921,93.762376
6,LLY,283.63,0.0,0.833741,88.712871,0.365179,91.089109,0.362392,98.217822,0.163996,96.831683,93.712871
7,LB,80.74,0.0,2.336642,100.000000,0.832176,99.801980,0.219641,91.881188,0.082858,82.574257,93.564356
8,NVDA,210.15,0.0,0.730728,83.762376,0.410424,95.049505,0.440321,99.603960,0.120412,91.683168,92.524752
9,IPG,37.92,0.0,1.156691,96.633663,0.471917,98.217822,0.113886,76.435644,0.166239,97.227723,92.128713


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [18]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [19]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format': '$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format': '0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format': '0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [23]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template],
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [24]:
writer.save()